In [ ]:
from matplotlib import animation, rc
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
import os
import gym
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)
env = gym.make("MountainCar-v0")

In [ ]:
import gym
import time
import optuna
import numpy as np
from blueqat import opt
Opt = opt.Opt

# なんか色々定義

In [ ]:
env = gym.make("MountainCar-v0")
qstr = "{}*(1 - (q0 + q1 + q2))^2"

In [ ]:
def get_matrix(position, velocity, a=5, b=1e-2, c=1):
    # 中心を0に近づけ、負の値を正にする
    e_p = abs(position + 0.5)**2

    matrix = [-1*(e_p+a*abs(velocity + b * -1)), -1*(e_p+a *
                                                     abs(velocity + b * 0)), -1*(e_p+a*abs(velocity + b * 1))]
    return matrix

In [ ]:
qaoa_action_dic = {(1, 0, 0): 0, (0, 1, 0): 1,
                   (0, 0, 1): 2, (1, 0): 0, (0, 1): 2}


def get_action(action_qaoa):
    return qaoa_action_dic[action_qaoa]


def get_qaoa(p, v, a, b, c):
    return Opt().add(np.diag(get_matrix(p, v, a, b))).add(qstr.format(c))


def get_max_position(obs):
    return max([(i[0] + 0.5)**2 for i in obs])

# optunaしてみる

In [ ]:
# 50回回して評価


def objective(trial):
    a = trial.suggest_loguniform("a", 1e1, 1e3)
    b = trial.suggest_uniform("b", 1e-2, 1e-1)
    c = trial.suggest_loguniform("c", 1e1, 1e3)

    observation = env.reset()
    missed_count = 0
    obs = []

    for i in range(50):
        p, v = observation

        problem = get_qaoa(p, v, a, b, c)

        action_qaoa = tuple(problem.run())
        while sum(action_qaoa) != 1:
            missed_count += 1
            if missed_count == 50:
                return 100
            action_qaoa = tuple(problem.run())

        action = get_action(action_qaoa)

        observation, reward, done, info = env.step(action)

        obs.append(observation)

        if done == True:
            print("finissed")
            print(obs)
            return -(51 - len(obs)) * 100

    return -100 * get_max_position(obs)

In [ ]:
study = optuna.create_study()

In [ ]:
start = time.time()
study.optimize(objective, n_trials=10, n_jobs=-1)
seconds = time.time() - start

In [ ]:
seconds

In [ ]:
study.best_params.values()

In [ ]:
a, b, c = study.best_params.values()

In [ ]:
observation = env.reset()

fig = plt.figure()
ims = []
actions = []
missed_count = 0
obs = []

for i in range(200):
    p, v = observation

    problem = get_qaoa(p, v, a, b, c)

    #action_qaoa = tuple(problem.run())
    action_qaoa = problem.qaoa().most_common()[0][0]
    while sum(action_qaoa) != 1:
        missed_count += 1
        if missed_count >= 200:
            break
        action_qaoa = tuple(problem.run())

    action = get_action(action_qaoa)

    actions.append(action)

    observation, reward, done, info = env.step(action)
    obs.append(observation)

    im = plt.imshow(env.render(mode='rgb_array'))
    ims.append([im])

    if done == True:
        break

In [ ]:
ani = animation.ArtistAnimation(fig, ims, interval=15, blit=True)
rc('animation', html='jshtml')
ani

In [ ]:
len(obs)

In [ ]:
missed_count